In [1]:
import numpy as np
import pandas as pd
from math import sqrt, log
from itertools import chain, product
from collections import defaultdict
%load_ext autotime


df1 = pd.read_excel('corpus.xlsx', sheetname='sh',index_col=0)
df2 = pd.read_excel('corpus.xlsx', sheetname='lo',index_col=0)

print(df1.head())
print(df2.head())

                                                      DESCRIPTION
UNIQUEID                                                         
BAR006FDC-ABAG  ACCUMULATORS PIPING AND VALVES AND OTHER RELAT...
BAR006FDC-AAAB                 HYDRAULIC FLUID CIRC. PUMP PUMPS C
BAR007WBA                                                       A
BAR007FCC                                                       C
BAR007FHC                                                       C
                                                             DESCRIPTION
UNIQUEID                                                                
BAR006-ASA-HV-21002A   U6A HRSG POWER AUGMENTATION STEAM MANUAL SUPPL...
BAR006-ASA-MCMA-1AB    U6A HRSG POWER AUGMENTATION STEAM BLOCKING VAL...
BAR006-ASA-MCMA-1AJ    U6A HRSG POWER AUGMENTATION VENT VALVE 480V BR...
BAR006-ASA-MCMB-10B    U6B POWER AUGMENTATION STEAM BLOCKING VALVE 48...
BAR006-ASA-PNV-21001A  U6A HRSG POWER AUGMENTATION PRESSURE CONTROL A...


In [2]:
df2.head()

,DESCRIPTION
UNIQUEID,
BAR006-ASA-HV-21002A,U6A HRSG POWER AUGMENTATION STEAM MANUAL SUPPL...
BAR006-ASA-MCMA-1AB,U6A HRSG POWER AUGMENTATION STEAM BLOCKING VAL...
BAR006-ASA-MCMA-1AJ,U6A HRSG POWER AUGMENTATION VENT VALVE 480V BR...
BAR006-ASA-MCMB-10B,U6B POWER AUGMENTATION STEAM BLOCKING VALVE 48...
BAR006-ASA-PNV-21001A,U6A HRSG POWER AUGMENTATION PRESSURE CONTROL A...


In [3]:
import numpy as np
import pandas as pd
from math import sqrt, log
from itertools import chain, product
from collections import defaultdict

# Load the main 2 tables I will be comparing
df1 = pd.read_excel('corpus.xlsx', sheetname='sh',index_col=0)
df2 = pd.read_excel('corpus.xlsx', sheetname='lo',index_col=0)

# method of calculating similarity of the one-hot encoding
def cosine_sim(u,v):
    return np.dot(u,v) / (sqrt(np.dot(u,u)) * sqrt(np.dot(v,v)))

# Vectorize to one-hot binary encoding as numbers work better than strings!
def corpus2vectors(corpus):
    def vectorize(sentence, vocab):
        return [sentence.split().count(i.lower()) for i in vocab]
    vectorized_corpus = []
    # Remobe dupes and sort
    vocab = sorted(set(chain(*[i.lower().split() for i in corpus])))
    for i in corpus:
        vectorized_corpus.append((i, vectorize(i, vocab)))
    return vectorized_corpus, vocab

# Basically just combine these two datasets to get a single corpus
def create_test_corpus(data1,data2):
    all_sents = list(data1) + list(data2)
    corpus, vocab = corpus2vectors(all_sents)
    return corpus, vocab      

# Workhouse function, cycle through all combinations and compute scores
def test_cosine(data1,data2, verbose=False):
    score,highestScore = 0,0
    counter = 0
    newMatch = []
    matches = []
    
    def create_test_corpus(corp1,corp2):
        
        data1_lwr = corp1['DESCRIPTION'].str.lower()
        data2_lwr = corp2['DESCRIPTION'].str.lower() 
        
        all_sents = list(data1_lwr) + list(data2_lwr)
        corpus, vocab = corpus2vectors(all_sents)
        return corpus, vocab, data1_lwr, data2_lwr
                       
    corpus,vocab,corp_1,corp_2 = create_test_corpus(corp1=data1,corp2=data2)
    
    corpus_dict = {}
    test = list(zip(*corpus))
    for counter,i in enumerate(test[0]):
        corpus_dict.update({test[0][counter]:test[1][counter]})
    
    #print("CORPUS")
    #print(corpus)
    # Cycle through each row in data1
    for dat1_ix, dat1 in enumerate(corp_1):
        if verbose == True:
            print("################################################################")
            print("dat1:",dat1)

        # For each iteration of data1, cycle through all of data2 below
        for dat2_ix, dat2 in enumerate(corp_2):
            
            if verbose == True:
                print("_______________________")
                print("Phrase_1:[{0}] {1}".format(dat1_ix,dat1))
                print("Phrase_2:[{0}] {1}".format(dat2_ix,dat2))
                print('length: phrase_1:',len(dat1),' phrase_2:',len(dat2))
            

            # Calculate cosime similarity from each line to the other
            score = cosine_sim(corpus_dict[dat1],corpus_dict[dat2])

            newMatch = [dat1_ix, dat2_ix, score]
            matches.append(newMatch)
        
        counter = counter + 1
    
    return matches

In [ ]:
from multiprocessing import Pool

# Run this with a pool of 5 agents having a chunksize of 3 until finished
df1_1 = df1.iloc[:1000,:]
df1_2 = df1.iloc[1000:2000,:]
df1_3 = df1.iloc[2000:3000,:]
df1_4 = df1.iloc[3000:4000,:]
df1_5 = df1.iloc[4000:5000,:]
df1_6 = df1.iloc[5000:6000,:]
df1_7 = df1.iloc[6000:,:]

go1 = [df1_1,df1_2,df1_3,df1_4,df1_5,df1_6,df1_7]

dataset = go1
agents = 7
chunksize = 1
with Pool(processes=agents) as pool:
    result = pool.map(test_cosine, dataset, chunksize)

In [13]:
# Run the main program, can be slow!
matches = test_cosine(df1,df2)

# Convert to DataFrame and grab top-250 per index (phrase) in document 1
dfMatches = pd.DataFrame(matches)
dfMatches.sort_values(2, ascending=False, inplace=True)
top250Matches = dfMatches.groupby(0).head(250)

# Join back the full phrases from the indices in columns 0,1
df1['newIndex'] = list(range(0, len(df1)))
df2['newIndex'] = list(range(0, len(df2)))
top250Matches = top250Matches.merge(df1,left_on=0,right_on='newIndex',how='left')
top250Matches = top250Matches.merge(df2,left_on=1,right_on='newIndex',how='left')

# Sort by original order from document 1 using index
top250Matches.sort_values(0, ascending=False, inplace=True)

# Clean up the DataFrame
top250Matches.drop(['newIndex_x','newIndex_y',0,1], axis=1, inplace=True)
top250Matches.columns.values[0] = 'CosineScore'

In [91]:
# Save to CSV if you want
top250Matches.to_csv("top250CosineScores.csv", index_label=False)

time: 16.3 s
